
Connect to Google Drive for datasets (colab)

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Install dependencies (colab)

In [2]:
#!pip install simpleitk

Dependencies

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories <Br>
Comment out directory not in use


In [2]:

#Toy Dataset Slices 
dummy_train_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")
dummy_train_label_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_label_slices")
dummy_test_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_test_img_slices")
dummy_test_label_slice_dir= pathlib.Path(r"spider_toy_dset_slices/dummy_test_label_slices")


Image Slice Class

In [3]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri_Slice:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case

        #grabbing the image as well for downsampling testing
        self.mri_mha = mri_mha
        #resample images to 64x64
       

        #get 2d array from mri slice
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array
        
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Zero crop sanity check

In [4]:
from transforms import array_transforms

test_image_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_img_slices/1_t1_30.mha")
test_label_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_label_slices/1_t1_30.mha")

print(test_label_slice.mri_mha)

test_image_a = test_image_slice.hu_a
test_label_a = test_label_slice.hu_a

test_image_a, test_label_a = array_transforms.crop_zero(test_image_a, test_label_a)

test_image_crop_mha = sitk.GetImageFromArray(test_image_a)

sitk.WriteImage(test_image_crop_mha, "D:/sitk dump/1_t1_30_cropzero.mha")

#TODO continue here make images 1:1 scale and downsample to 64x64
#if(sitk_slice.GetSize()[0] != sitk_slice.GetSize()[1])

Image (000002A00552D030)
  RTTI typeinfo:   class itk::Image<double,2>
  Reference Count: 1
  Modified Time: 2067
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2044
  UpdateMTime: 2063
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [467, 482]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [467, 482]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [467, 482]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (000002A00C69DF70)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,double>
      Reference Count: 1
      Modified Time: 2060
      Debug: Off
      Object Name: 
      Observers: 
        none


Sort directories


In [5]:
#get lists from directories

#toy dset slices
image_path = dummy_train_img_slice_dir
label_path = dummy_train_label_slice_dir

image_dir_list = os.listdir(image_path)
label_dir_list = os.listdir(label_path)

print(image_path) 
print(label_path)

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(label_path))

print(dirlen)

#print(local_label_idr)


spider_toy_dset_slices\dummy_train_img_slices
spider_toy_dset_slices\dummy_train_label_slices
2100


Get max dimension of slice in dset for x y padding <br>
Images have slices with 0 label info removed and are cropped using zero crop <br>
Also get min max values of dset for tensor normalization 

In [6]:
from transforms import array_transforms



row_list = []
col_list = []

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = image_path.joinpath(image_dir_list[idx])
  lbl_path = label_path.joinpath(label_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of hte file 
  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of hte file 
  '''
  image = Mri_Slice(img_path)
  label = Mri_Slice(lbl_path)

  image_a = image.hu_a
  label_a = label.hu_a

  #crop zero
  image_a_cropzero, label_a_cropzero = array_transforms.crop_zero(image_a, label_a)

  #resampling code will go here instead of crop-zero


  if(idx ==0):
    image_tensor_min = np.min(image_a_cropzero)
    image_tensor_max = np.max(label_a_cropzero)
    label_tensor_min = np.min(label_a_cropzero)
    label_tensor_max = np.max(label_a_cropzero)

    unique_masks, masks_counts = np.unique(label_a, return_counts=True)
  else:
    if(np.min(image_a_cropzero) < image_tensor_min):
      image_tensor_min = np.min(image_a_cropzero)
      image_tensor_min_dir = img_path
    if(np.min(label_a_cropzero) < label_tensor_min):
      label_tensor_min = np.min(label_a_cropzero)
    if(np.max(image_a_cropzero) > image_tensor_max):
      image_tensor_max = np.max(image_a_cropzero)
    if(np.max(label_a_cropzero) > label_tensor_max):
      label_tensor_max = np.max(label_a_cropzero)

    current_masks, current_mask_counts = np.unique(label_a, return_counts=True)
    if(len(current_masks > len(unique_masks))):
      unique_masks = current_masks
  #print("image res", image_a_cropzero.shape)
  
  
  row_list.append(image_a_cropzero.shape[0]) #add row value to list
  #print(image_a_cropzero.shape[0])
  col_list.append(image_a_cropzero.shape[1]) #add col value to list 
  

#calculate max 
row_dim_max = max(row_list)
col_dim_max = max(col_list)

row_dim_max = ((row_dim_max + 15) // 16) * 16 #nearest multiple of 16
col_dim_max = ((col_dim_max + 15) // 16) * 16 #nearest multiple of 16

print("row max:", max(row_list))
print("col max:", max(col_list))

print("image tensor min", image_tensor_min)
print("image tensor max", image_tensor_max)
print("label tensor min", label_tensor_min)
print("label tensor max", label_tensor_max)

print("amount of masks", unique_masks)





row max: 448
col max: 224
image tensor min -2057.0
image tensor max 4292.0
label tensor min 0.0
label tensor max 208.0
amount of masks [2.]


Dataset Class

In [7]:
from transforms import tensor_transforms

class SpiderDataset(Dataset):
    def __init__(self, labels_dir, img_dir, transform=None, target_transform=None):
        self.labels_dir = labels_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.labels_dir))

    def __getitem__(self, idx):
        label_dir_list = os.listdir(self.labels_dir)
        image_dir_list = os.listdir(self.img_dir)

        image_dir_list = natsorted(image_dir_list)
        label_dir_list = natsorted(label_dir_list)

        img_path = self.img_dir.joinpath(image_dir_list[idx])
        label_path = self.labels_dir.joinpath(label_dir_list[idx])

        image = Mri_Slice(img_path)
        label = Mri_Slice(label_path)

        image_a = image.hu_a
        label_a = label.hu_a

        #comment out the part not being used whether for 3d or 2d model 
    
        '''
        #3d tensor for 3D CNN
        image_tensor = torch.from_numpy(image.hu_a)
        label_tensor = torch.from_numpy(label.hu_a)
        '''

        image_tensor = torch.from_numpy(image_a)
        label_tensor = torch.from_numpy(label_a)
        
        image_tensor = image_tensor.to(torch.float32)
        label_tensor = label_tensor.to(torch.float32)
        
        #pad to max resolution of slice in dset 
        image_tensor = tensor_transforms.pad_to_resolution(image_tensor, [row_dim_max, col_dim_max])
        label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max])

        
         #normalise
        image_tensor = (image_tensor - image_tensor_min) / (image_tensor_max - image_tensor_min)
        label_tensor = (label_tensor - label_tensor_min) / (label_tensor_max - label_tensor_min)

        #comment out unsqueeze testing
        
        image_tensor = image_tensor.unsqueeze(0)
        label_tensor = label_tensor.unsqueeze(0)
        

        image_tensor = image_tensor.to(device)
        label_tensor = label_tensor.to(device)

        #print(image_tensor.shape)
        return image_tensor, label_tensor




Dataset Classes


In [8]:
#toy train test dataset to test network running
#local_train_set = SpiderDataset(local_img_idr, local_label_idr)
dummy_train_set = SpiderDataset(dummy_train_label_slice_dir, dummy_train_img_slice_dir)

dummy_test_set = SpiderDataset(dummy_test_label_slice_dir, dummy_test_img_slice_dir)

print("train dataset len",dummy_train_set.__len__())
print("test dataset len",dummy_test_set.__len__())

train dataset len 2100
test dataset len 215


Create Unet Instance







In [10]:
from models import unet 

input_channels = 1 #Hounsfield scale
output_channels = 3 #Vertebra, disc and spinal canal masks SHOULD BE 3 FOR 3 MASKS
output_channels = 18 #one for every mask
model = unet.UNet(in_channels= 1,num_classes=18)
model.to(device)
model.to(torch.float32)
#for param in model.parameters():
 #   print(param.device)

d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:211: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:212: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


UNet(
  (conv_final): Conv2d(64, 18, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): DownConv(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): DownConv(
      (conv1): Conv2d(256, 512, kernel_size=(3,

Create Faster-RCNN Instance trained


In [11]:
'''
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes

weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
'''

'\nfrom torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights\nfrom torchvision.utils import draw_bounding_boxes\n\nweights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT\nmodel = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)\n'

Create Mask RCNN Instance Trained


In [12]:
'''
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights

model = maskrcnn_resnet50_fpn_v2(
    weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,
    num_classes = 19
    #weights_backbone = 
)
'''

'\nfrom torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights\n\nmodel = maskrcnn_resnet50_fpn_v2(\n    weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,\n    num_classes = 19\n    #weights_backbone = \n)\n'

Hyperparameters


In [13]:
epochs = 25 #testing
lr = 0.001 #testing
batchsize = 8 #testing
loss_func = nn.L1Loss() #testing
loss_func.to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)

print(epochs)
print(lr)
print(batchsize)
print(loss_func)



25
0.001
8
L1Loss()


Dataloader

In [14]:
dummy_train_dataloader = DataLoader(dummy_train_set, batch_size = batchsize, shuffle=True)

dummy_test_dataloader = DataLoader(dummy_test_set, batch_size = batchsize, shuffle=True)

'''
for batch in dummy_train_dataloader:
    for tensor in batch:
        print("min", torch.min(tensor))
        print("max", torch.max(tensor))
'''

'\nfor batch in dummy_train_dataloader:\n    for tensor in batch:\n        print("min", torch.min(tensor))\n        print("max", torch.max(tensor))\n'

Tensor Dimensions Check

In [ ]:
'''
for idx_slice in dummy_train_dataloader:
    for tensor in idx_slice:
        print(tensor.shape)
        break
'''

'\nfor idx_slice in dummy_train_dataloader:\n    for tensor in idx_slice:\n        print(tensor.shape)\n        break\n'

One Epoch <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [15]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    #swap train dataloader for dset
    for i, data in enumerate(dummy_train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        #print("labels", labels.shape)
        #print(inputs.shape)
        #print(device)

        #inputs = inputs.transpose(-1,0)
        #labels = labels.transpose(-1,0)
        #inputs = inputs.reshape(inputs.shape(1), inputs.shape(0), inputs.shape(3), inputs.shape(4))
        #labels = labels.reshape(labels.shape(1), labels.shape(0), labels.shape(3), labels.shape(4))

      
        # Zero your gradients for every batch!
        optim.zero_grad()

        #2d
        # Make predictions for this batch
        outputs = model(inputs)

        #print("outputs", outputs.shape)

        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optim.step()

        #3d tensor, probably won't use keeping here just in case  
        '''
        #run everything while indexing through z axis
        for axis in images, masks:
          for idx in range(0, axis.size(1)):
            # Zero your gradients for every batch!
            optim.zero_grad()
            print(inputs[:, idx, : ,:])
            # Make predictions for this batch
            outputs = model(inputs[:, idx, : ,:])

            # Compute the loss and its gradients
            loss = loss_func(outputs[:, idx, : ,:], labels[:, idx, : ,:])
            loss.backward()

            # Adjust learning weights
            optim.step()
        '''
        # Gather data and report
        running_loss += loss.item()
        #if i % 1000 == 999:
            #print("goes in")
        last_loss = running_loss / 1000 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        tb_x = epoch_index * len(dummy_train_dataloader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.
        #if ends here

    print("loss", loss)
    return last_loss

Train Loop <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html


In [16]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/spider_seg_{}'.format(timestamp))
epoch_number = 0


best_vloss = 1_000_000.

for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    print("avg loss in epoch", avg_loss)

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(dummy_test_dataloader):
            vinputs, vlabels = vdata

            voutputs = model(vinputs)
            vloss = loss_func(voutputs, vlabels)
            running_vloss += vloss
            

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        #commented out saving the model for now to debug loss being 0 
        '''
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
        '''
    epoch_number += 1

EPOCH 1:


d:\Software\Anaconda\envs\spider-torch-tb\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([8, 1, 448, 224])) that is different to the input size (torch.Size([8, 18, 448, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


  batch 1 loss: 6.562002003192902e-05
  batch 2 loss: 3.455359861254692e-05
  batch 3 loss: 3.8962133228778837e-05
  batch 4 loss: 3.423799574375152e-05
  batch 5 loss: 3.369764983654022e-05
  batch 6 loss: 3.413935750722885e-05
  batch 7 loss: 2.305922657251358e-05
  batch 8 loss: 1.5446368604898453e-05
  batch 9 loss: 2.7574608102440833e-05
  batch 10 loss: 3.4116242080926895e-05
  batch 11 loss: 3.030323050916195e-05
  batch 12 loss: 2.356870472431183e-05
  batch 13 loss: 1.4294401742517948e-05
  batch 14 loss: 2.9359165579080582e-05
  batch 15 loss: 4.8859580419957635e-06
  batch 16 loss: 2.0409954711794854e-05
  batch 17 loss: 2.938254177570343e-05
  batch 18 loss: 2.566668763756752e-05
  batch 19 loss: 2.2052083164453505e-05
  batch 20 loss: 1.624944433569908e-05
  batch 21 loss: 1.493249461054802e-05
  batch 22 loss: 1.4035378582775592e-05
  batch 23 loss: 4.4594258069992067e-05
  batch 24 loss: 2.2740108892321588e-05
  batch 25 loss: 1.9362663850188256e-05
  batch 26 loss: 2.87

d:\Software\Anaconda\envs\spider-torch-tb\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([4, 1, 448, 224])) that is different to the input size (torch.Size([4, 18, 448, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


  batch 263 loss: 5.48645406961441e-05
loss tensor(0.0549, device='cuda:0', grad_fn=<MeanBackward0>)
avg loss in epoch 5.48645406961441e-05


d:\Software\Anaconda\envs\spider-torch-tb\Lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([7, 1, 448, 224])) that is different to the input size (torch.Size([7, 18, 448, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


LOSS train 5.48645406961441e-05 valid 0.02233799360692501
EPOCH 2:
  batch 1 loss: 1.1163328774273395e-05
  batch 2 loss: 7.898879237473011e-06
  batch 3 loss: 1.3629998080432415e-05
  batch 4 loss: 7.30373477563262e-06
  batch 5 loss: 2.3469734936952592e-05
  batch 6 loss: 1.3231482356786728e-05
  batch 7 loss: 2.5037627667188645e-05
  batch 8 loss: 3.529488667845726e-05
  batch 9 loss: 1.4321001246571542e-05
  batch 10 loss: 3.4038886427879336e-05
  batch 11 loss: 2.4277226999402045e-05
  batch 12 loss: 2.5705128908157347e-05
  batch 13 loss: 2.1584665402770043e-05
  batch 14 loss: 1.6660537570714952e-05
  batch 15 loss: 1.9044650718569756e-05
  batch 16 loss: 3.3453863114118574e-05
  batch 17 loss: 2.9272077605128287e-05
  batch 18 loss: 1.1556564830243587e-05
  batch 19 loss: 2.0482828840613366e-05
  batch 20 loss: 1.9980723038315772e-05
  batch 21 loss: 2.3285554721951484e-05
  batch 22 loss: 2.2344574332237244e-05
  batch 23 loss: 3.773067519068718e-05
  batch 24 loss: 2.92394012

KeyboardInterrupt: 